##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Data augmentation

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/data_augmentation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотреть на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/images/data_augmentation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустить в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/images/data_augmentation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Смотреть исходные файлы на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/images/data_augmentation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом уроке будет рассмотрено увеличение данных(аугментация): техника, позволяющай увеличить ваш тренировочный датасет с помощью случайных(но реалистичных) трансформаций изображения, например поворот. Вы изучиет два возможных способа аугментации данных. Первый - с использованием [Keras Preprocessing Layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/). Второй - используя `tf.image`.

## Импорт Tensorflow

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

## Загрузка датасета

В этом уроке используется датасет [tf_flowers](https://www.tensorflow.org/datasets/catalog/tf_flowers). Для удобства загрузите набор данных с помощью [TensorFlow Datasets](https://www.tensorflow.org/datasets). Если вы хотите узнать о других способах импорта данных, см. [load images](https://www.tensorflow.org/tutorials/load_data/images).


In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

Датасет "flowers" имеет 5 классов.

In [ ]:
num_classes = metadata.features['label'].num_classes
print(num_classes)

Давайте возьмем изображение из датасета, чтобы использовать его для демонстрации аугментации.

In [ ]:
get_label_name = metadata.features['label'].int2str

image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

## Использования слоя предобработки Keras(preprocessing layers)

Примечание. Слои [Keras Preprocesing Layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing), представленные в этом разделе, в настоящее время являются экспериментальными.

### Изменение размера и масштабирование


Вы можете использовать слои предварительной обработки, чтобы [изменить размер изображений](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Resizing) и [масштабировать изображения](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling).

In [ ]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

Примечание. Слой масштабирования выше нормализует значения пикселей до `[0,1]`. Если вместо этого вы хотите нормализовать до `[-1,1]`, нужно передать аргумент *offset = -1*: Rescaling (1./127.5, offset = -1).

Вы можете просмотреть результат применения этих слоев на изображение

In [ ]:
result = resize_and_rescale(image)
_ = plt.imshow(result)

Вы также можете убедится, что пиксели находятся в диапазоне `[0-1]`.

In [ ]:
print("Min and max pixel values:", result.numpy().min(), result.numpy().max())

### Аугментация данных

Вы также можете использовать слои предварительной обработки для аугментации данных.

Давайте создадим несколько слоев предварительной обработки и повторно применим их к одному и тому же изображению.

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
# Добавим изображение в пакет
image = tf.expand_dims(image, 0)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0])
  plt.axis("off")

Существует множество вариантов [предварительной обработки](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing), которые вы можете использовать для аугментации данных. Например слои: `layers.RandomContrast`, `layers.RandomCrop`, `layers.RandomZoom` и другие.

### Два варианта использования слоев предварительной обработки

Существует два способа использования слоев предварительной обработки, но с важными замечаниями.

#### Способ 1: Сделать слой предварительной обработки частью вашей модели

In [ ]:
model = tf.keras.Sequential([
  resize_and_rescale,
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  # Остальная часть вашей модели
])

В этом случае следует помнить о двух важных моментах:

* Аугментация данных будет выполняться на устройстве синхронно с остальными вашими слоями и будет использовать
 графический процессор.

* Когда вы экспортируете вашу модель с помощью `model.save`, слои предварительной обработки будут сохранены вместе с остальной частью вашей модели. Если вы позже развернете эту модель, она автоматически нормализует изображения(в соответствии с конфигурацией ваших слоев). Это может избавить вас от необходимости заново реализовывать эту логику на стороне сервера.

Примечание. Увеличение данных не работает во время тестирования, поэтому входные изображения будут аугментироваться только во время вызовов `model.fit` (не `model.evaluate` или `model.predict`).

#### Способ 2: Применить слои предварительной обработки к датасету

In [ ]:
aug_ds = train_ds.map(
  lambda x, y: (resize_and_rescale(x, training=True), y))

При таком подходе вы используете `Dataset.map` для создания пакетов аугментированных изображений. 
В этом случае:
* Увеличение данных происходит асинхронно на ЦП и не является блокирующим. Вы можете частично перекрывать обучение вашей модели на GPU с предварительной обработкой данных, используя `Dataset.prefetch`, как будет показано ниже.
* В этом случае слои предварительной обработки не будут экспортированы вместе с моделью при вызове `model.save`. Вам нужно будет прикрепить их к вашей модели перед ее сохранением или повторной реализацией на стороне сервера. После обучения вы можете прикрепить слои предварительной обработки перед экспортом.

Вы можете найти пример первого варианта в учебнике [классификация изображений](https://www.tensorflow.org/tutorials/images/classification). Здесь продемонстрируем второй вариант.

### Применение слоев предварительной обработки к датасету

Вы настроите датасеты для обучения, проверки и тестирования с помощью слоев предварительной обработки, которые вы создали выше. Также вы сконфигурируете датасеты для повышения производительности, используя параллельное чтение и буферизованную предварительную выборку, чтобы читать пакеты с диска без блокировки ввода-вывода. Вы можете узнать больше о производительности датасета в руководстве [Повышение производительности с помощью tf.data API](https://www.tensorflow.org/guide/data_performance).

Примечание: аугментация данных должна применяться только к тренировочному датасету.

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  # Изменение размера и масштабирование всех наборов данных
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Пакетная обработка всех наборов данных
  ds = ds.batch(batch_size)

  # Используем аугментацию данных только на обучающем датасете
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Используем буферизованную предварительную выборку для всех наборов данных
  return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

### Обучение модели

В завершение, обучите модель, используя эти наборы данных. Эта модель не была настроена на точность (цель данного урока - показать вам механику).

In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
loss, acc = model.evaluate(test_ds)
print("Accuracy", acc)

### Пользовательская аугментация данных

Вы также можете создавать собственные слои аугментации данных. В этом руководстве показаны два способа. 
1. Сначала вы создадите слой `layers.Lambda`. Это хороший способ написать лаконичный код. 
2. Затем вы напишете новый слой с помощью [наследования классов](https://www.tensorflow.org/guide/keras/custom_layers_and_models), это даст вам больше контроля. Оба слоя будут случайным образом инвертировать цвета изображения с некоторой вероятностью. 

In [ ]:
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (255-x)
  else:
    x
  return x

In [ ]:
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))

random_invert = random_invert()

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = random_invert(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0].numpy().astype("uint8"))
  plt.axis("off")

Теперь реализуем настраиваемый слой с помощью [наследования класса](https://www.tensorflow.org/guide/keras/custom_layers_and_models).

In [ ]:
class RandomInvert(layers.Layer):
  def __init__(self, factor=0.5, **kwargs):
    super().__init__(**kwargs)
    self.factor = factor

  def call(self, x):
    return random_invert_img(x)

In [ ]:
_ = plt.imshow(RandomInvert()(image)[0])

Оба эти слоя можно использовать, как описано в вариантах 1 и 2 выше.

## Использование tf.image

Вышеупомянутые утилиты `layers.preprocessing` удобны, однако для более точного управления вы можете написать свои собственные конвейеры или слои аугментации, используя tf.data и tf.image. Вы также можете ознакомиться с [TensorFlow Addons Image: Operations](https://www.tensorflow.org/addons/tutorials/image_ops) и [TensorFlow I/O: преобразование цветового пространства](https://www.tensorflow.org/io/tutorials/colorspace)

Поскольку датасет цветов ранее был аугментирован, давайте повторно импортируем его, чтобы начать все заново.

In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

Получим изображение для работы.

In [ ]:
image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

Для визуализации и сравнения исходных и дополненных изображений воспользуемся следующей функцией.

In [ ]:
def visualize(original, augmented):
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image')
  plt.imshow(original)

  plt.subplot(1,2,2)
  plt.title('Augmented image')
  plt.imshow(augmented)

### Аугментация данных

### Переворачивание изображения

Отразим изображение по вертикали или горизонтали.

In [ ]:
flipped = tf.image.flip_left_right(image)
visualize(image, flipped)

### Преобразование в серое

Преобразование в изображение с оттенками серого

In [ ]:
grayscaled = tf.image.rgb_to_grayscale(image)
visualize(image, tf.squeeze(grayscaled))
_ = plt.colorbar()

### Насыщение изображение

Увеличим насыщенность цветов изображения, задав коэффициент насыщенности.

In [ ]:
saturated = tf.image.adjust_saturation(image, 3)
visualize(image, saturated)

### Изменение яркости изображения

Изменим яркость изображения, указав коэффициент яркости.

In [ ]:
bright = tf.image.adjust_brightness(image, 0.4)
visualize(image, bright)

### Обрезка изображения по центру

Обрежем изображение от центра до желаемой части изображения.

In [ ]:
cropped = tf.image.central_crop(image, central_fraction=0.5)
visualize(image,cropped)

### Поворот изображения

Повернем изображение на 90 градусов.

In [ ]:
rotated = tf.image.rot90(image)
visualize(image, rotated)

### Применение аугментации к датасету

Как и раньше, применим аугментацию данных к набору данных с помощью `Dataset.map`.

In [ ]:
def resize_and_rescale(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
  image = (image / 255.0)
  return image, label

In [ ]:
def augment(image,label):
  image, label = resize_and_rescale(image, label)
  # Add 6 pixels of padding
  image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6) 
   # Random crop back to the original size
  image = tf.image.random_crop(image, size=[IMG_SIZE, IMG_SIZE, 3])
  image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
  image = tf.clip_by_value(image, 0, 1)
  return image, label

### Настройка датасетов

In [ ]:
train_ds = (
    train_ds
    .shuffle(1000)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
) 

In [ ]:
val_ds = (
    val_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
test_ds = (
    test_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

Эти датасеты теперь можно использовать для обучения модели, как показано ранее.

## Что дальше?

Это руководство показало вам как аугментировать данные с помощью [Keras Preprocessing Layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/) и `tf.image`. Чтобы узнать как включить слои предобработки в вашу модель - смотрите руководство по [Image classification](https://www.tensorflow.org/tutorials/images/classification). Вам также может быть интересно узнать, как слои предварительной обработки могут помочь в классификации текста - [Basic text classification](https://www.tensorflow.org/tutorials/keras/text_classification). Вы можете узнать больше о `tf.data` в этом [руководстве](https://www.tensorflow.org/guide/data), и вы можете узнать, как настроить входные конвейеры для повышения производительности [здесь](https://www.tensorflow.org/guide/data_performance).